# Downloading data from Quesmed

Direct DB connection to download the latest comments

In [1]:
import psycopg2
from dotenv import load_dotenv
import os

load_dotenv()

conn = psycopg2.connect(
  dbname=os.getenv("QUESMED_POSTGRES_DB"),
  user=os.getenv("QUESMED_POSTGRES_USER"),
  password=os.getenv("QUESMED_POSTGRES_PASSWORD"),
  port=os.getenv("QUESMED_POSTGRES_PORT"),
  host=os.getenv("QUESMED_POSTGRES_HOST")
)

In [2]:
keys = (
  'id', 
  'createdAt', 
  'userId', 
  'userCreatedAt', 
  'classYear', 
  'universityId',
  'country',
  'universityName',
  'parentId',
  'questionId',
  'comment',
  'review',
  'likes',
  'dislikes'
)
data_map = {k: [] for k in keys}
data_map

{'id': [],
 'createdAt': [],
 'userId': [],
 'userCreatedAt': [],
 'classYear': [],
 'universityId': [],
 'country': [],
 'universityName': [],
 'parentId': [],
 'questionId': [],
 'comment': [],
 'review': [],
 'likes': [],
 'dislikes': []}

In [4]:
import pandas as pd
from tqdm import tqdm

file_path = "../data/1comments.h5"

def load_df():
  if os.path.isfile(file_path):
    df = pd.read_hdf(file_path, key='df')
    return df

  cur = conn.cursor()
  cur.execute("""
SELECT
  com.id,
  com."createdAt",
  com."userId",
  u."createdAt" "userCreatedAt",
  u."classYear",
  u."universityId",
  uni.country,
  uni.name "universityName",
  com."parentId",
  com."questionId",
  com.comment,
  com.review,
  sum(qcl."likeTrueDislikeFalse"::int) "likes",
  sum(abs(qcl."likeTrueDislikeFalse"::int - 1)) "dislikes"
FROM
  question_comments com
  INNER JOIN users u ON com."userId" = u.id
  INNER JOIN universities uni ON u."universityId" = uni.id
  LEFT JOIN question_comment_likes qcl ON com.id = qcl."commentId"
    AND qcl."likeTrueDislikeFalse" IS NOT NULL
GROUP BY
  com.id,
  u.id,
  uni.id
ORDER BY
  com."createdAt" ASC;
  """)

  t = tqdm()
  batch_size = 1000
  loading = True
  while loading:
      data = cur.fetchmany(batch_size)
      t.update(len(data))
      if len(data) < batch_size:
          loading = False
      for row in data:
          for i, k in enumerate(keys):
              data_map[k].append(row[i])
  t.close()
  cur.close()

  df = pd.DataFrame.from_dict(data_map)
  df.to_hdf(file_path, key='df', mode='w')
  return df

df = load_df()
print(df.shape)
df.sample(3)

30312it [00:00, 510312.57it/s]


(30312, 14)


/var/folders/lk/lhmn07j501l6rng06tjrl8cm0000gn/T/ipykernel_69621/2883840197.py:57: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['classYear', 'country', 'universityName', 'comment', 'review'], dtype='object')]

  df.to_hdf(file_path, key='df', mode='w')


,id,createdAt,userId,userCreatedAt,classYear,universityId,country,universityName,parentId,questionId,comment,review,likes,dislikes
1876,2943,2021-05-31 12:55:15.201000+00:00,4816,2021-02-24 14:39:33.720000+00:00,Year 4,2620,United Kingdom,University College London (UCL),NaN,6266,am I a surgeon please,False,18.0,1.0
37,119,2021-03-29 12:22:11.633000+00:00,1195,2019-03-18 18:41:10.891000+00:00,Year 4,2620,United Kingdom,University College London (UCL),NaN,4281,You don't know what this guy could be upto ;),False,20.0,0.0
10902,14952,2022-11-30 14:18:50.935558+00:00,11020,2021-08-09 20:12:25.340000+00:00,Year 4,2637,United Kingdom,University of East Anglia,NaN,8981,"NIPE is done at 6w, he is 6m old? when is this...",True,NaN,NaN
